In [2]:
from Bio import Entrez
import pandas as pd
from xml.dom.minidom import parse
import xml.dom.minidom
import pickle
import json
from pymed import PubMed
import requests
import datetime
import os

pubmed = PubMed(tool="MyTool", email="jefflovejoy1314@gmail.com")
# Entrez.email = "Your.Name.Here@example.org"
Entrez.email = 'jefflovejoy1314@gmail.com'
handle = Entrez.esearch(db="pubmed", term="breast cancer",retmode="xml")
record = Entrez.read(handle)
handle.close()


In [ ]:
# Download necessary data
all_mesh_terms = pd.read_csv('mesh_for_Diseases.csv')
downloaded_names = []
for r,d,f in os.walk('./Mesh_terms'):
    for name in f:
        downloaded_names.append(name.split('.csv')[0])
all_mesh_names = list(all_mesh_terms['Name'])

# downloaded_names
remaining = [i for i in all_mesh_names if ' '.join(i.split(',')) not in downloaded_names]

# n.authors, n.conclusions,n.doi,n.journal,n.keywords,n.publication_date,n.pubmed_id,n.title
# {'pubmed_id': [], 'authors': [(author1 (first + last), affiliation)], 'keywords':[()], abstract':[],'publication_date':[],'title':[],'results':[],'doi':[], 'journal':[]}


for mesh_name in remaining:
    query_string = mesh_name
    results = pubmed.query(query_string, max_results=10000)

    info_dict = {'pubmed_id': [], 'authors': [], 'keywords':[], 'abstract':[],'publication_date':[],'title':[],'results':[],'doi':[], 
                 'journal':[]}

    for i in results: 
        for key in list(info_dict.keys()):
            try:
                if key == 'authors':
                    au = i.authors
                    auinfo = []
                    for x in au: 
                        name = str(x['firstname']) + ' ' + str(x['lastname'])
                        affiliation = x['affiliation']
                        auinfo.append((name,affiliation))
                    info_dict[key].append(auinfo)
                elif key == 'pubmed_id':
                    info_dict[key].append(str(i.pubmed_id.split('\n')[0]))
                else:
                    info_dict[key].append(getattr(i,key))

            except:
                info_dict[key].append(None)


    query_string= ' '.join(query_string.split(','))
    pd.DataFrame(info_dict).to_csv(f'Mesh_terms/{query_string}.csv')


In [ ]:
# download cancer literature

query_string = '("neoplasms"[MeSH Terms] OR "neoplasms"[All Fields] OR "cancer"[All Fields]) AND ("2009/11/24"[PDat] : "2019/11/21"[PDat])'
results = pubmed.query(query_string, max_results=3965593)


file_number = 0

info_dict = {'pubmed_id': [], 'authors': [], 'keywords':[], 'abstract':[],'publication_date':[],'title':[],'results':[],'doi':[], 
             'journal':[]}

for i in results:
    print(len(info_dict['pubmed_id']))
    for key in list(info_dict.keys()):
        try:
            if key == 'authors':
                au = i.authors
                auinfo = []
                for x in au: 
                    name = str(x['firstname']) + ' ' + str(x['lastname'])
                    affiliation = x['affiliation']
                    auinfo.append((name,affiliation))
                info_dict[key].append(auinfo)
            elif key == 'pubmed_id':
                info_dict[key].append(str(i.pubmed_id.split('\n')[0]))
            else:
                info_dict[key].append(getattr(i,key))

        except:
            info_dict[key].append(None)
            
    if len(info_dict['pubmed_id']) == 15000:
        pd.DataFrame(info_dict).to_csv(f'cancer/cancer_{file_number}.csv')
        info_dict = {'pubmed_id': [], 'authors': [], 'keywords':[], 'abstract':[],'publication_date':[],'title':[],'results':[],'doi':[], 
             'journal':[]}
        file_number += 1

        print(file_number)
        

In [ ]:
info_dict = {'pubmed_id': [], 'authors': [], 'keywords':[], 'abstract':[],'publication_date':[],'title':[],'results':[],'doi':[], 
             'journal':[]}

for i in results:
    for key in list(info_dict.keys()):
        try:
            if key == 'authors':
                au = i.authors
                auinfo = []
                for x in au: 
                    name = str(x['firstname']) + ' ' + str(x['lastname'])
                    affiliation = x['affiliation']
                    auinfo.append(f'{name}-{affiliation}')
                info_dict[key].append(';'.join(auinfo))
            elif key == 'pubmed_id':
                info_dict[key].append(str(i.pubmed_id.split('\n')[0]))
            else:
                info_dict[key].append(getattr(i,key))

        except:
            info_dict[key].append(None)
    
    break
            

In [ ]:
# future work
# result.json().get('linksets')[0]['linksetdbs'][0]['links']

# taks1
#     write help function to parse citations and type of article
#   https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&id=29233583 summary 
# citation
# result = requests.get('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pubmed_pmc_refs&id=21876726&id=21876761',params= {'retmode':'json'})

In [ ]:
# parse type of article
result = requests.get('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pubmed_pmc_refs&id=21876726&id=21876761',params= {'retmode':'json'})

result2 = requests.get('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pubmed_pmc_refs&id=29233583',params= {'retmode':'json'})

result = requests.get('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&id=29233583',params= {'retmode':'json'})
pubtype = result.json()['result']['29233583']['pubtype']


# parse type of article -- example 
pubmed_id = 29233583
request_type = requests.get(f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&id={pubmed_id}',params= {'retmode':'json'})
citation_request = requests.get(f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pubmed_pmc_refs&id={pubmed_id}',params= {'retmode':'json'})
pubtype = request_type.json()['result']['29233583']['pubtype']

print(len(citation_request.json().get('linksets')[0]['linksetdbs'][0]['links']))

